## Working with LLM programatically: LangChain Router Pattern
Router pattern is a way to direct a user’s query to the most appropriate sub-chain or tool based on the input, instead of sending all queries to the same model or prompt.


## Step 1 : Set the project path

In [1]:
# set the working directory for the project
%cd /home/vcap/app/cf-jupyterlab-workshop

/home/vcap/app/cf-jupyterlab-workshop


## Step 2: Import the dependencies

In [4]:
import sys, os
import requests
import json
import warnings
from openai import OpenAI
import httpx
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_classic.chains.router import MultiPromptChain
from langchain_classic.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain_core.prompts import PromptTemplate
from cfenv import AppEnv
from tanzu_utils import CFGenAIService
warnings.filterwarnings('ignore')

## Step 3: Load the credentials for the model

In [5]:
# load your service details replace name with your Gen AI service.  Gen AI service is bound to the app
genai = CFGenAIService("tanzu-gpt-oss-120b")

# List available models
models = genai.list_models()
for m in models:
    print(f"- {m['name']} (capabilities: {', '.join(m['capabilities'])})")

# construct chat_credentials
chat_credentials = {
    "api_base": genai.api_base + "/openai/v1",
    "api_key": genai.api_key,
    "model_name": models[0]["name"]
}

- openai/gpt-oss-120b (capabilities: CHAT, TOOLS)


 ## Step 4: initialize langchain chat model with credentials

In [6]:
# 2. HTTP client (optional but recommended for custom config)
httpx_client = httpx.Client(verify=False)  # verify=False if your endpoint needs --insecure

# 3. Initialize the LLM
llm = ChatOpenAI(
    temperature=0.9,
    model=chat_credentials["model_name"],   # model name from CF service
    base_url=chat_credentials["api_base"],  # OpenAI-compatible endpoint
    api_key=chat_credentials["api_key"],    # Bearer token
    http_client=httpx_client
)

## Step 5: Define prompt templates

In [7]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

## Step 6: Build Destination Chains and Default Chains.  
This section dynamically constructs multiple LLM processing chains (called "destination chains") based on a list of prompt definitions.

In [8]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

## Step 7: Define multi part router prompt template

In [9]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

## Step 8: Define router template, router prompt and router chain
This section constructs a *router chain* — a specialized LLM-based
decision component that determines which destination chain should
handle a given user input.

In [10]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

## Step 9: Execute the router chain

In [11]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'**Black‑body radiation** is the electromagnetic radiation that a perfect absorber (and therefore also a perfect emitter) of light would give off when it is in thermal equilibrium at some temperature\u202f\\(T\\).\n\n---\n\n### Key ideas\n\n| Concept | What it means |\n|---------|---------------|\n| **Perfect absorber** | A “black body” absorbs **all** incident radiation, regardless of wavelength or direction. Because nothing is reflected, it looks completely black when cold. |\n| **Thermal equilibrium** | The object’s temperature is constant; the energy it absorbs from its surroundings equals the energy it emits. |\n| **Emission spectrum** | The radiation it emits depends only on its temperature, not on its material or shape. It has a characteristic shape that peaks at a wavelength that shifts with temperature. |\n\n---\n\n### The spectrum\n\nThe spectral energy density (energy per unit volume per unit frequency) of a black body is given by **Planck’s law**  \n\n\\[\nu(\\nu,T)=\\frac{